**This code will grab the extents of the RTCs you downloaded in the previous folder, as well as the temporal boundaries of your dataset. It will download Total Precipitations from ERA5 for the desired area. BE AWARE: depending on EMCWF's current requests number, this can take from several minutes to several hours**.

In [ ]:
import cdsapi
import xarray as xr
import datetime
from ipyfilechooser import FileChooser
import rioxarray as rxr
import xarray as xr
import fierpy
import glob
import pandas as pd
from pathlib import Path
from ipyfilechooser import FileChooser
import re
from fier_local import reof as freof
from fier_local import sel_best_fit

import opensarlab_lib as asfn
from osgeo import gdal, osr
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.dates as mdates

**Function to grab the dates of the RTCs**

In [ ]:
def get_dates(dir_path, prefix):
    dates = []
    pths = list(dir_path.glob(f'{prefix}.tif*'))

    for p in pths:
        date_regex = '\d{8}'
        date = re.search(date_regex, str(p))
        if date:
            dates.append(date.group(0))
    return dates


               
def get_coordinates_from_geotiff_bbox(geotiff_path):
    dataset = gdal.Open(geotiff_path)

    # Get the GeoTransform to convert pixel coordinates to geographical coordinates
    transform = dataset.GetGeoTransform()

    # Get the raster size (number of rows and columns)
    cols = dataset.RasterXSize
    rows = dataset.RasterYSize

    # Get the four corner points in pixel coordinates
    corners = [(0, 0), (cols, 0), (cols, rows), (0, rows)]

    # Calculate the longitude (X) and latitude (Y) of each corner
    corner_coordinates = []
    for corner in corners:
        lon = transform[0] + corner[0] * transform[1] + corner[1] * transform[2]
        lat = transform[3] + corner[0] * transform[4] + corner[1] * transform[5]

        # Get the spatial reference system of the dataset
        srs = osr.SpatialReference()
        srs.ImportFromWkt(dataset.GetProjection())

        # Create a coordinate transformation object
        target_srs = osr.SpatialReference()
        target_srs.ImportFromEPSG(4326)  # EPSG code for WGS84
        transform_obj = osr.CoordinateTransformation(srs, target_srs)

        # Transform the coordinates to WGS84 (latitude and longitude)
        lon, lat, _ = transform_obj.TransformPoint(lon, lat)

        corner_coordinates.append((lat, lon))
        
    corner_coordinates = np.array(corner_coordinates)
    boundaries = np.array([[np.min(corner_coordinates[:,1]), np.min(corner_coordinates[:,0])],
                           [np.max(corner_coordinates[:,1]), np.max(corner_coordinates[:,0])]])

    return boundaries


#### **Select the parent folder cointing your RTCs' folder**

In [ ]:
fc = FileChooser(Path.cwd())
display(fc)

**Grab the spatial and temporal extents of the tiffs**

In [ ]:
# Load the paths to the tiff files
main_dir = Path(str(fc.selected))
tiff_dir = list(main_dir.glob('*tiffs'))[0]
tiffs = glob.glob(f"{str(tiff_dir)}/*tif")


# Get the coordinates of the AOI
geotiff_path = str(tiffs[0])
boundaries = get_coordinates_from_geotiff_bbox(geotiff_path)
for idx, corner in enumerate(boundaries, 1):
    lat, lon = corner
    print(f"Corner {idx}: Latitude: {lat}, Longitude: {lon}")

times = get_dates(tiff_dir, '*VV')
times.sort()
times = pd.DatetimeIndex(times)
times.name = "time"

**Send a quiery to EMCWF to grab the ERA5 precipitation data for the spatial and temporal extent of our tiffs.**

In [ ]:

# Create a folder to host the ERA5 data
weather_data_folder = Path(fc.selected)/'ERA5'
weather_data_folder.mkdir(exist_ok=True)


# Create a new instance of the CDS API client
# Define the API URL and key
api_url = 'https://cds.climate.copernicus.eu/api/v2'
api_key = '105035:a341bca0-1659-4056-a656-f4d505fffd4c'


# Create the API client
c = cdsapi.Client(url=api_url, key=api_key)

# Create a list of the years covered by the tiffs
years = [str(i) for i in np.arange(times.min().year, times.max().year+1)]

# Get the data from ECMWF
c.retrieve(
        'reanalysis-era5-single-levels',
        {
            'product_type': 'reanalysis',
            'variable': 'total_precipitation',
            'format': 'netcdf',
            'year': years,
            'month': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
            ],
            'day': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
                '13', '14', '15',
                '16', '17', '18',
                '19', '20', '21',
                '22', '23', '24',
                '25', '26', '27',
                '28', '29', '30',
                '31',
            ],
            'time': [
                '00:00', '01:00', '02:00',
                '03:00', '04:00', '05:00',
                '06:00', '07:00', '08:00',
                '09:00', '10:00', '11:00',
                '12:00', '13:00', '14:00',
                '15:00', '16:00', '17:00',
                '18:00', '19:00', '20:00',
                '21:00', '22:00', '23:00',
            ],
            'area': [boundaries[0,0], boundaries[0,1], boundaries[1,0], boundaries[1,1]],  # AOI
        },
        f'{weather_data_folder}/era5_data.nc',
)

# Load precipitation dataset's name
filename = f'{weather_data_folder}/era5_data.nc'

# Load it, drop the 2nd level of precipitation (expver=5), resample for 1 value per day, in-between are summed, and save as a netcdf
ds = xr.open_dataset(filename).resample(time='D').sum().to_netcdf(f'{weather_data_folder}/era5_data.nc')
